# Otimização de Portfólio de Investimentos

**Disciplina**: Otimização - 2025.2

**Professor**: Luidi Simonetti

**Alunos**: Bernardo Pozzato e Pedro Cintra

---

## 1. Introdução

### 1.1 Motivação
No cenário financeiro atual, a diversificação é uma das ferramentas mais poderosas para a gestão de risco. No entanto, a construção de um portfólio eficiente — que maximize o retorno para um dado nível de risco — não é uma tarefa trivial. A simples alocação ingênua (pesos iguais) raramente produz resultados ótimos.

A motivação deste trabalho é aplicar métodos quantitativos avançados para resolver o problema de alocação de ativos, utilizando tanto técnicas exatas (Programação Quadrática Inteira Mista - MIQP) quanto meta-heurísticas (Algoritmos Genéticos) para lidar com restrições complexas do mundo real.

### 1.2 Definição do Problema
O problema consiste em selecionar a proporção ideal de capital a ser alocada em um conjunto de ativos pré-selecionados (ações, fundos imobiliários, criptomoedas e renda fixa). O objetivo é encontrar o ponto ótimo na fronteira eficiente que satisfaça as preferências de risco do investidor e respeite diversas restrições operacionais (como limites de exposição por setor e por ativo).

## 2. Modelo Matemático (Detalhado)

Adotamos uma extensão do modelo de Média-Variância de Markowitz, enriquecido com penalidades adicionais para métricas fundamentais (P/VP) e de risco de cauda (CVaR).

### 2.1 Variáveis de Decisão
Seja $N$ o número de ativos no universo de investimento. Definimos:

*   **$w \in \mathbb{R}^N$**: Vetor de pesos contínuos ($w_i$ é a fração do capital no ativo $i$).
*   **$z \in \{0, 1\}^N$**: Vetor de variáveis binárias ($z_i = 1$ se o ativo $i$ for escolhido, $0$ caso contrário). *Usado apenas no Gurobi para modelagem MIQP.*

### 2.2 Parâmetros do Modelo
*   $\mu \in \mathbb{R}^N$: Vetor de retornos esperados anuais.
*   $\Sigma \in \mathbb{R}^{N \times N}$: Matriz de covariância dos retornos.
*   $\lambda \in \mathbb{R}^+$: Coeficiente de aversão ao risco do investidor.
*   $PVP \in \mathbb{R}^N$: Vetor de Preço/Valor Patrimonial dos ativos.
*   $CVaR \in \mathbb{R}^N$: Vetor de Conditional Value at Risk (95%) dos ativos.
*   $\alpha_{PVP}, \alpha_{CVaR}, \alpha_{Caixa} \in \mathbb{R}^+$: Pesos de penalização na função objetivo.

### 2.3 Função Objetivo
O problema é formulado como uma minimização de uma função de custo composta:

$$ \min_{w} Z = \underbrace{\lambda (w^T \Sigma w)}_{\text{Penalidade de Risco}} - \underbrace{w^T \mu}_{\text{Retorno Esperado}} + \underbrace{\alpha_{PVP} (w^T PVP)}_{\text{Penalidade Valuation}} + \underbrace{\alpha_{CVaR} (w^T CVaR)}_{\text{Penalidade Risco Cauda}} + \underbrace{\alpha_{Caixa} \max(0, 1 - \sum_{i=1}^N w_i)}_{\text{Penalidade Caixa}} $$

**Interpretação dos Termos:**
*   **$\lambda (w^T \Sigma w)$**: Penaliza a variância (risco) do portfólio. Quanto maior $\lambda$, mais conservador.
*   **$- w^T \mu$**: Incentiva o retorno esperado (minimizar o negativo = maximizar).
*   **$\alpha_{PVP} (w^T PVP)$**: Penaliza ativos com alto P/VP (caros fundamentalmente).
*   **$\alpha_{CVaR} (w^T CVaR)$**: Penaliza ativos com alto risco de cauda (perdas extremas).
*   **$\alpha_{Caixa} \max(0, 1 - \sum w_i)$**: Penaliza fortemente deixar dinheiro não investido.

### 2.4 Restrições

#### 2.4.1 Restrições Comuns (GA e Gurobi)

1.  **Orçamento (Budget Constraint)**:
    $$ \sum_{i=1}^{N} w_i \le 1 $$
    *Permitimos que a soma seja menor que 1, mas penalizamos fortemente via função objetivo.*

2.  **Não-Negatividade (Long Only)**:
    $$ w_i \ge 0, \quad \forall i \in \{1, ..., N\} $$

3.  **Limite Individual por Ativo (Asset Cap)**:
    $$ w_i \le w_{max}, \quad \forall i \in \{1, ..., N\} $$
    *Tipicamente $w_{max} = 0.30$ (30%).*

4.  **Limite por Setor (Sector Cap)**:
    Seja $S_k$ o conjunto de índices de ativos pertencentes ao setor $k$:
    $$ \sum_{j \in S_k} w_j \le w_{setor}, \quad \forall k $$

5.  **Teto de Volatilidade (Risk Constraint)**:
    $$ \sqrt{w^T \Sigma w} \le \sigma_{max} $$
    *Equivalente a: $w^T \Sigma w \le \sigma_{max}^2$*

#### 2.4.2 Restrição de Peso Mínimo (Cardinality)

Para evitar "poeira" (pesos insignificantes), exigimos que se um ativo for escolhido, ele tenha um peso mínimo relevante.

**Algoritmo Genético (Heurística):**
Implementado via pós-processamento (Repair Operator):
```python
X[X < 0.005] = 0.0  # Zera pesos < 0.5%
```

**Gurobi (Modelagem Exata - MIQP):**
Usando variáveis binárias $z_i$ e a formulação Big-M:
$$ 0.005 \cdot z_i \le w_i \le w_{max} \cdot z_i, \quad \forall i $$

**Interpretação:**
*   Se $z_i = 0 \implies 0 \le w_i \le 0 \implies w_i = 0$ (ativo não escolhido)
*   Se $z_i = 1 \implies 0.005 \le w_i \le w_{max}$ (ativo escolhido com peso mínimo de 0.5%)

Esta formulação transforma o problema do Gurobi em um **MIQP (Mixed-Integer Quadratic Programming)**.

## 3. Solução Computacional

Utilizamos duas abordagens para resolver este problema:

### 3.1 Algoritmo Genético (NSGA-II / pymoo)
Uma meta-heurística populacional que evolui soluções através de operadores genéticos (cruzamento, mutação) e um operador de reparo customizado para garantir viabilidade.

**Vantagens:**
*   Lida bem com não-linearidades e descontinuidades
*   Flexível para adicionar restrições complexas
*   Não requer convexidade

**Desvantagens:**
*   Não garante ótimo global
*   Tempo de execução pode ser alto

### 3.2 Solver Exato (Gurobi - MIQP)
Um solver comercial de otimização que resolve o problema utilizando Branch-and-Bound para as variáveis inteiras e métodos de ponto interior para a parte quadrática.

**Vantagens:**
*   Garante ótimo global (ou prova que não existe)
*   Altamente otimizado e eficiente
*   Fornece bounds de otimalidade

**Desvantagens:**
*   Requer licença (acadêmica disponível)
*   Limitado a problemas convexos/quadráticos
*   Variáveis binárias aumentam complexidade computacional

Abaixo, carregamos as configurações e executamos os otimizadores.

In [ ]:
import sys
import os

# Adiciona o diretório atual ao path para importar os módulos
sys.path.append(os.getcwd())

import config
import preparar_dados
import otimizar
import otm_gurobi_setores
import pandas as pd

# Configurações da Simulação
VALOR_INVESTIR = 10000
LAMBDA_RISCO = 50.0  # Aversão ao risco moderada/alta
RISCO_MAXIMO = 0.15  # 15% a.a.
TETO_ATIVO = 0.30    # 30% por ativo
TETO_SETOR = 1.00    # Sem restrição forte de setor

print("--- Carregando Dados ---")
inputs = preparar_dados.calcular_inputs_otimizacao(VALOR_INVESTIR)
print("Dados carregados com sucesso.")

In [ ]:
print("\n--- Executando Algoritmo Genético ---")
res_ga = otimizar.rodar_otimização(
    inputs, 
    RISCO_MAXIMO, 
    LAMBDA_RISCO, 
    setores_proibidos=[], 
    teto_maximo_ativo=TETO_ATIVO, 
    teto_maximo_setor=TETO_SETOR
)

if res_ga:
    print(f"GA Concluído. Retorno: {res_ga['metricas']['retorno_aa']:.2f}% | Risco: {res_ga['metricas']['risco_aa']:.2f}%")
    print(f"Objetivo (GA): {res_ga['metricas']['score']:.6f}")

In [ ]:
print("\n--- Executando Solver Gurobi (MIQP) ---")
res_gurobi = otm_gurobi_setores.resolver_com_gurobi_setores(
    inputs, 
    LAMBDA_RISCO,
    RISCO_MAXIMO, 
    warm_start_pesos=None,
    setores_proibidos=[], 
    teto_maximo_ativo=TETO_ATIVO, 
    teto_maximo_setor=TETO_SETOR
)

if res_gurobi:
    print(f"Gurobi Concluído. Retorno: {res_gurobi['retorno']*100:.2f}% | Risco: {res_gurobi['risco']*100:.2f}%")
    print(f"Objetivo (Gurobi): {res_gurobi['obj']:.6f}")

## 4. Discussão dos Resultados

### 4.1 Comparação GA vs. Gurobi (MIQP)

**Função Objetivo:**
Ambos os métodos otimizam a **mesma função objetivo**:
$$ \min \left( \lambda \sigma^2 - \mu + \alpha_{PVP} \cdot PVP + \alpha_{CVaR} \cdot CVaR + \alpha_{Caixa} \cdot Caixa \right) $$

**Expectativa Teórica:**
Como o Gurobi é um solver exato para problemas MIQP, ele deve encontrar o **ótimo global** (ou provar que não existe solução viável). Portanto, esperamos que:
$$ \text{Objetivo}_{Gurobi} \le \text{Objetivo}_{GA} $$

**Observações Práticas:**
*   O Gurobi tipicamente encontra soluções com valores de objetivo **menores** (melhores) que o GA.
*   O GA pode encontrar soluções competitivas em tempo menor, especialmente para problemas de grande escala.
*   A restrição de cardinalidade (peso mínimo) é tratada de forma **exata** no Gurobi (via variáveis binárias) e de forma **heurística** no GA (via operador de reparo).

### 4.3 Impacto das Penalidades
*   **P/VP**: Favorece ativos "baratos" fundamentalmente, reduzindo exposição a bolhas.
*   **CVaR**: Reduz exposição a riscos de cauda (eventos extremos), aumentando robustez.
*   **Caixa**: Força alocação completa do capital, evitando ineficiência de deixar dinheiro parado.

Esses termos adicionais transformam o problema de Mean-Variance puro em um modelo mais realista e robusto para o mercado brasileiro.